In [1]:
import gradio as gr
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
from torch import nn

import cv2
import numpy as np

from skimage.metrics import structural_similarity as ssim

import lpips


device='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using: {device}')

import warnings
warnings.filterwarnings("ignore")

Using: cuda


In [2]:
def calculate_psnr(img1, img2):
    # Convert images to float32
    img1 = img1.astype(np.float32)
    img2 = img2.astype(np.float32)
    
    # Compute MSE
    mse = np.mean((img1 - img2) ** 2)
    
    if mse == 0:
        return float('inf')
    
    # Calculate PSNR
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    
    return psnr

def calculate_ssim(img1, img2):
    # Convert images to grayscale (if needed)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM
    ssim_value, _ = ssim(img1_gray, img2_gray, full=True)
    
    return ssim_value

def calculate_lpips(img1, img2):
    # Create the LPIPS metric
    loss_fn = lpips.LPIPS(net='alex', verbose=False)
    
    # Convert images to PyTorch tensors
    img1_tensor = lpips.im2tensor(img1)
    img2_tensor = lpips.im2tensor(img2)
    
    # Calculate LPIPS distance
    lpips_value = loss_fn(img1_tensor, img2_tensor)
    
    return lpips_value.item()

def compare_metrics(hr_img, sr_img, lr_img):
    # Calculate metrics for HR vs SR
    psnr_hr_sr = calculate_psnr(hr_img, sr_img)
    ssim_hr_sr = calculate_ssim(hr_img, sr_img)
    lpips_hr_sr = calculate_lpips(hr_img, sr_img)
    
    # Calculate metrics for HR vs LR
    psnr_hr_lr = calculate_psnr(hr_img, lr_img)
    ssim_hr_lr = calculate_ssim(hr_img, lr_img)
    lpips_hr_lr = calculate_lpips(hr_img, lr_img)
    
    # Calculate metrics for HR vs Bicubic SR (for comparison)
    bicubic_sr_img = cv2.resize(lr_img, (hr_img.shape[1], hr_img.shape[0]), interpolation=cv2.INTER_CUBIC)
    psnr_hr_bicubic = calculate_psnr(hr_img, bicubic_sr_img)
    ssim_hr_bicubic = calculate_ssim(hr_img, bicubic_sr_img)
    lpips_hr_bicubic = calculate_lpips(hr_img, bicubic_sr_img)
    
    # Print results
    print("Metrics for HR vs SR (proposed method):")
    print(f"PSNR: {psnr_hr_sr:.2f} dB, SSIM: {ssim_hr_sr:.4f}, LPIPS: {lpips_hr_sr:.4f}")
    print("Metrics for HR vs SR (bicubic):")
    print(f"PSNR: {psnr_hr_lr:.2f} dB, SSIM: {ssim_hr_lr:.4f}, LPIPS: {lpips_hr_lr:.4f}")

# Example usage:
hr_image_path = '../Sample_2.jpg'
sr_image_path = '../sample2_correction_model_results/Upscaled Image 914x609_mobileSR.png'  # Super-resolved image
lr_image_path = '../experiment_detection/solo_2/Upscaled Image 914x609.png'  # Low-resolution image

hr_img = cv2.imread(hr_image_path)
sr_img = cv2.imread(sr_image_path)
lr_img = cv2.imread(lr_image_path)

compare_metrics(hr_img, sr_img, lr_img)

Metrics for HR vs SR (proposed method):
PSNR: 28.95 dB, SSIM: 0.7582, LPIPS: 0.3794
Metrics for HR vs SR (bicubic):
PSNR: 27.23 dB, SSIM: 0.6684, LPIPS: 0.2878


In [3]:
sr_image_path = '../sample2_correction_model_results/Upscaled Image 914x609_miniSRGAN.png'  # Super-resolved image
sr_img = cv2.imread(sr_image_path)

compare_metrics(hr_img, sr_img, lr_img)

Metrics for HR vs SR (proposed method):
PSNR: 27.41 dB, SSIM: 0.7154, LPIPS: 0.7205
Metrics for HR vs SR (bicubic):
PSNR: 27.23 dB, SSIM: 0.6684, LPIPS: 0.2878


In [4]:
sr_image_path = '../sample2_correction_model_results/Upscaled Image 914x609_EDSR.png'  # Super-resolved image
sr_img = cv2.imread(sr_image_path)

compare_metrics(hr_img, sr_img, lr_img)

Metrics for HR vs SR (proposed method):
PSNR: 29.19 dB, SSIM: 0.7657, LPIPS: 0.6461
Metrics for HR vs SR (bicubic):
PSNR: 27.23 dB, SSIM: 0.6684, LPIPS: 0.2878
